In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('..')
import random, math
import pyzx as zx
from fractions import Fraction
from pyzx.phasepoly import circuit_phase_polynomial_blocks, optimize_block
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\scala.jar'

In [3]:
import os

class CircuitComparer:
    def __init__(self, dirname, before, after):
        self.fname_before = os.path.join(dirname, before)
        self.fname_after = os.path.join(dirname, after)
        self.name = before[:-7]
        self.has_run = False
    
    def __str__(self):
        return "CircuitComparer({}, {})".format(self.name, str(self.has_run))

    def __repr__(self):
        return str(self)
    
    def run(self):
        if self.has_run: return True
        try: 
            c = zx.Circuit.from_quipper_file(self.fname_after).to_basic_gates()
            self.t_opt = c.tcount()
            self.qubits = c.qubits
            c = zx.Circuit.from_quipper_file(self.fname_before).to_basic_gates()
        except TypeError: return False
        self.gatecount = len(c.gates)
        self.t_before = c.tcount()
        g = c.to_graph()
        zx.simplify.full_reduce(g)
        self.t_after = zx.tcount(g)
        self.has_run = True
        del c, g
        return True
    
    def pretty(self):
        if not self.has_run:
            success = self.run()
        else: success = True
        if not success: 
            return self.name + "    -"
        s = self.name.ljust(20) + str(self.qubits).rjust(7)
        s += str(self.gatecount).rjust(8) + str(self.t_before).rjust(7) + str(self.t_opt).rjust(9) + str(self.t_after).rjust(7)
        return s

In [4]:

dirs = [r'..\circuits\Arithmetic_and_Toffoli', r'..\circuits\QFT_and_Adders']
beforefiles = []
afterfiles = []
for d in dirs:
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1:
            beforefiles.append((f,d))
        else: afterfiles.append((f,d))

circuits = []
for f, d in beforefiles:
    n = f[:-7]
    for f2,d2 in afterfiles:
        if d!=d2: continue
        #n = f2.replace("_after_heavy","")
        if f2.startswith(n):
            c = CircuitComparer(d, f, f2)
            circuits.append(c)
            break
#circuits[:30]

In [5]:
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-kitchen", "T-us")
for c in circuits:
    if c.run():
        print(c.pretty())

Circuit              qubits G-count T-before T-kitchen T-us
adder_8                  24    1014    399      215    179
barenco_tof_10           19     514    224      100    100
barenco_tof_3             5      66     28       16     16
barenco_tof_4             7     130     56       28     28
barenco_tof_5             9     194     84       40     40
csla_mux_3_original      15     190     70       64     62
csum_mux_9_corrected     30     448    196       84     84
gf2^E10_mult             30    1709    700      410    410
gf2^E16_mult             48    4397   1792     1040   1040
gf2^E4_mult              12     275    112       68     68
gf2^E5_mult              15     429    175      115    115
gf2^E6_mult              18     617    252      150    150
gf2^E7_mult              21     839    343      217    217
gf2^E8_mult              24    1109    448      264    264
gf2^E9_mult              27    1385    567      351    351
mod5_4                    5      71     28       16    

In [16]:
c = [c for c in circuits if c.name == 'adder_8'][0]
g = zx.Circuit.from_quipper_file(c.fname_before).to_graph()
zx.full_reduce(g, quiet=False)
g.normalise()
#zx.draw(g,figsize=(20,3))
c2 = zx.extract.streaming_extract(g).to_basic_gates()
print(len(c2.gates), c2.tcount())
#zx.draw(c2.to_graph())
# t = c2.to_tensor()
# c3 = zx.Circuit.from_quipper_file(c.fname_after)
# g = c3.to_graph()
# zx.full_reduce(g)
# g.normalise()
# c4 = zx.extract.streaming_extract(g).to_basic_gates()
# zx.draw(c4.to_graph())
# t2 = c3.to_tensor()
# print("break")
# print(c3.gates, c3.tcount())
#zx.compare_tensors(c2.to_tensor(),c4.to_tensor())

spider_simp: 171. 129. 76. 37. 23. 14. 8. 5. 3. 1.  10 iterations
id_simp: 108. 57.  2 iterations
spider_simp: 21. 2.  2 iterations
pivot_simp: 31. 4. 2. 1. 1.  5 iterations
lcomp_simp: 6. 6.  2 iterations
id_simp: 5.  1 iterations
spider_simp: 4.  1 iterations
pivot_simp: 1.  1 iterations
lcomp_simp: 3. 3.  2 iterations
id_simp: 1.  1 iterations
spider_simp: 1.  1 iterations
Gadgetizing...
pivot_simp: 144. 51. 37. 35. 21. 11. 14. 11. 10. 7. 7. 7. 4. 3. 1.  15 iterations
Back to clifford_simp
id_simp: 41.  1 iterations
spider_simp: 13. 12. 5. 3. 2. 1.  6 iterations
pivot_simp: 9. 9. 7. 5. 1. 1.  6 iterations
id_simp: 29.  1 iterations
spider_simp: 13. 8. 3. 1. 1. 1.  6 iterations
pivot_simp: 1. 1.  2 iterations
lcomp_simp: 11. 9. 1. 3.  4 iterations
id_simp: 12.  1 iterations
spider_simp: 6. 4. 1. 1.  4 iterations
pivot_simp: 6. 3.  2 iterations
id_simp: 4.  1 iterations
spider_simp: 4.  1 iterations
lcomp_simp: 4. 4.  2 iterations
id_simp: 2.  1 iterations
spider_simp: 2.  1 iteration

MemoryError: 

In [90]:
count

42

In [37]:
print("Circuit".ljust(30), "qubits", "G-count", "T-count", "T-optimized")
for c in circuits:
    g = c.to_graph()
    t_before = zx.t_count(g)
    zx.simplify.full_reduce(g)
    t_after = zx.t_count(g)
    print(c.name.ljust(30), str(c.qubits).rjust(6), str(len(c.gates)).rjust(7), str(t_before).rjust(7), str(t_after).rjust(7))

Circuit                        qubits G-count T-count T-optimized
adder_8_before                     24    1014     399 179
barenco_tof_10_before              19     514     224 100
barenco_tof_3_before                5      66      28 16
barenco_tof_4_before                7     130      56 28
barenco_tof_5_before                9     194      84 40
csla_mux_3_before_original         15     190      70 62
csum_mux_9_before_corrected        30     448     196 84
gf2^E10_mult_before                30    1709     700 410
gf2^E16_mult_before                48    4397    1792 1040
gf2^E4_mult_before                 12     275     112 68
gf2^E5_mult_before                 15     429     175 115
gf2^E6_mult_before                 18     617     252 150
gf2^E7_mult_before                 21     839     343 217
gf2^E8_mult_before                 24    1109     448 264
gf2^E9_mult_before                 27    1385     567 351
mod5_4_before                       5      71      28 8
mod_adder_102

In [46]:
c = [c for c in circuits if c.name.find('Adder8')!=-1][0]
g = c.to_graph()
zx.full_reduce(g, quiet=True)
zx.t_count(g)
#g.normalise()
#zx.draw(g)

56